## Celda 1 - Checklist Conforme a los Archivos ```NPY```

In [ ]:
import numpy as np
from pathlib import Path

BASE = Path(r"C:\Users\leona\Documents\Thesis_Project_UACH\Temp\Dataset\features_mfcc_labeled")

X_train = np.load(BASE / "X_train.npy", mmap_mode="r")
y_train = np.load(BASE / "y_train.npy")
X_val   = np.load(BASE / "X_val.npy",   mmap_mode="r")
y_val   = np.load(BASE / "y_val.npy")
X_test  = np.load(BASE / "X_test.npy",  mmap_mode="r")
y_test  = np.load(BASE / "y_test.npy")

print("X_train:", X_train.shape, X_train.dtype)
print("y_train:", y_train.shape, y_train.dtype)
print("X_val:  ", X_val.shape, X_val.dtype)
print("X_test: ", X_test.shape, X_test.dtype)

print("\nDistribución y_train:")
unique, counts = np.unique(y_train, return_counts=True)
print(dict(zip(unique, counts)))

## Celda 2 - CNN baseline (TensorFlow/Keras)

### Celda 2.1 - Dataset por batches (para no cargar todo a RAM)

In [ ]:
import tensorflow as tf

BATCH = 64

def make_ds(X, y, training=False):
    ds = tf.data.Dataset.from_tensor_slices((X, y))
    if training:
        ds = ds.shuffle(20000, reshuffle_each_iteration=True)
    ds = ds.batch(BATCH).prefetch(tf.data.AUTOTUNE)
    return ds

train_ds = make_ds(X_train, y_train, training=True)
val_ds   = make_ds(X_val, y_val, training=False)
test_ds  = make_ds(X_test, y_test, training=False)

### Celda 2.2 - CNN pequeña (baseline)

from tensorflow.keras import layers, models

num_classes = 4
input_shape = (3, 32, 201)  # (C, H, W) como lo guardaste

model = models.Sequential([
    layers.Input(shape=input_shape),
    # Keras por default usa channels_last; forzamos channels_first
    layers.Conv2D(16, (3,3), padding="same", activation="relu", data_format="channels_first"),
    layers.BatchNormalization(axis=1),
    layers.MaxPool2D((2,2), data_format="channels_first"),

    layers.Conv2D(32, (3,3), padding="same", activation="relu", data_format="channels_first"),
    layers.BatchNormalization(axis=1),
    layers.MaxPool2D((2,2), data_format="channels_first"),

    layers.Conv2D(64, (3,3), padding="same", activation="relu", data_format="channels_first"),
    layers.BatchNormalization(axis=1),
    layers.GlobalAveragePooling2D(data_format="channels_first"),

    layers.Dense(64, activation="relu"),
    layers.Dropout(0.3),
    layers.Dense(num_classes, activation="softmax")
])

model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-3),
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)

model.summary()

### Celda 2.3 - Entrenar

In [ ]:
callbacks = [
    tf.keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True),
    tf.keras.callbacks.ReduceLROnPlateau(patience=2, factor=0.5)
]

history = model.fit(train_ds, validation_data=val_ds, epochs=30, callbacks=callbacks)

### Celda 2.4 - Evaluar + matriz de confusión

In [ ]:
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix

y_pred = np.argmax(model.predict(test_ds), axis=1)
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred, digits=4))

## Celda 3 - Red Neuronal

### Celda 3.1 - Cargar ```.npy``` con Memmap

In [ ]:
import numpy as np
from pathlib import Path

BASE = Path(r"C:\Users\leona\Documents\Thesis_Project_UACH\Temp\Dataset\features_mfcc_labeled")

X_train = np.load(BASE / "X_train.npy", mmap_mode="r")
y_train = np.load(BASE / "y_train.npy")
X_val   = np.load(BASE / "X_val.npy",   mmap_mode="r")
y_val   = np.load(BASE / "y_val.npy")
X_test  = np.load(BASE / "X_test.npy",  mmap_mode="r")
y_test  = np.load(BASE / "y_test.npy")

print("X_train:", X_train.shape, X_train.dtype)
print("y_train:", y_train.shape, y_train.dtype)
print("X_val:  ", X_val.shape, X_val.dtype)
print("X_test: ", X_test.shape, X_test.dtype)

# Chequeo de clases
u, c = np.unique(y_train, return_counts=True)
print("Distribución train:", dict(zip(u, c)))

### Celda 3.2 - ```tf.data``` (con cast a float32 dentro del pipeline)

In [ ]:
import tensorflow as tf

BATCH = 64

def make_ds(X, y, training=False):
    ds = tf.data.Dataset.from_tensor_slices((X, y))
    if training:
        ds = ds.shuffle(20000, reshuffle_each_iteration=True)
    ds = ds.batch(BATCH)
    ds = ds.map(lambda a,b: (tf.cast(a, tf.float32), b), num_parallel_calls=tf.data.AUTOTUNE)
    ds = ds.prefetch(tf.data.AUTOTUNE)
    return ds

train_ds = make_ds(X_train, y_train, training=True)
val_ds   = make_ds(X_val, y_val, training=False)
test_ds  = make_ds(X_test, y_test, training=False)

### Celda 3.3 - Class weights (para el desbalance)

Esto ayuda mucho para que no “adivine todo clase 3” (clase predominante).

In [ ]:
import numpy as np

num_classes = 4
counts = np.bincount(y_train, minlength=num_classes)
total = counts.sum()

# Peso inverso a frecuencia (simple y efectivo)
class_weight = {i: float(total / (num_classes * counts[i])) for i in range(num_classes) if counts[i] > 0}
print("counts:", counts)
print("class_weight:", class_weight)

### Celda 3.4 - Modelo CNN baseline (channels_first)

In [ ]:
from tensorflow.keras import layers, models

input_shape = (3, 32, 201)

model = models.Sequential([
    layers.Input(shape=input_shape),

    layers.Conv2D(16, (3,3), padding="same", activation="relu", data_format="channels_first"),
    layers.BatchNormalization(axis=1),
    layers.MaxPool2D((2,2), data_format="channels_first"),

    layers.Conv2D(32, (3,3), padding="same", activation="relu", data_format="channels_first"),
    layers.BatchNormalization(axis=1),
    layers.MaxPool2D((2,2), data_format="channels_first"),

    layers.Conv2D(64, (3,3), padding="same", activation="relu", data_format="channels_first"),
    layers.BatchNormalization(axis=1),
    layers.GlobalAveragePooling2D(data_format="channels_first"),

    layers.Dense(64, activation="relu"),
    layers.Dropout(0.3),
    layers.Dense(4, activation="softmax")
])

model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-3),
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)

model.summary()

### Celda 3.5 - Entrenar (con callbacks)

In [ ]:
callbacks = [
    tf.keras.callbacks.EarlyStopping(patience=6, restore_best_weights=True),
    tf.keras.callbacks.ReduceLROnPlateau(patience=2, factor=0.5),
    tf.keras.callbacks.ModelCheckpoint(
        filepath=str(BASE / "cnn_mfcc_best.keras"),
        monitor="val_accuracy",
        save_best_only=True
    )
]

history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=40,
    class_weight=class_weight,
    callbacks=callbacks
)

### Celda 3.6 - Evaluación en Test (matriz + macro-F1)

In [ ]:
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix

probs = model.predict(test_ds)
y_pred = np.argmax(probs, axis=1)

print("Confusion matrix:\n", confusion_matrix(y_test, y_pred))
print("\nClassification report:\n", classification_report(y_test, y_pred, digits=4))